In [16]:
path_test_x="/Users/vishaal/Downloads/archive-5/t10k-images.idx3-ubyte"
path_test_y="/Users/vishaal/Downloads/archive-5/t10k-labels.idx1-ubyte"
path_train_x="/Users/vishaal/Downloads/archive-5/train-images.idx3-ubyte"
path_train_y="/Users/vishaal/Downloads/archive-5/train-labels.idx1-ubyte"

In [17]:
import struct
# this library is used for dealing with binary data
#will be using this after i convert the dat ain binary
import pandas as pd
import numpy as np

In [18]:
def load_mnist_images(filename):
    with open(filename, 'rb') as f:#rb opens file to read in binary format
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        images = np.fromfile(f, dtype=np.uint8).reshape(num, rows, cols)
    return images
def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        labels = np.fromfile(f, dtype=np.uint8)
    return labels
def preprocess_data(images, labels):
    images = images.reshape(images.shape[0], -1) / 255.0  # Normalize images
    labels_one_hot = np.zeros((labels.size, labels.max() + 1))
    labels_one_hot[np.arange(labels.size), labels] = 1 
    return images, labels_one_hot

In [19]:
#load and convert data
train_images=load_mnist_images(path_train_x)
train_labels = load_mnist_labels(path_train_y)
test_images = load_mnist_images(path_test_x)
test_labels = load_mnist_labels(path_test_y)

X_train, y_train = preprocess_data(train_images, train_labels)
X_test, y_test= preprocess_data(test_images, test_labels)

In [38]:
print(X_train.shape)
print(y_train.shape)#it is aldready hot encoded
dat=pd.DataFrame(y_train)
dat.dropna()
dat.shape
#no nan values

(60000, 784)
(60000, 10)


(60000, 10)

In [29]:
m,n=X_train.shape
#we aldrady have devided data
X_train.shape

(60000, 784)

In [30]:
def one_hot_to_digit(one_hot):
    return np.argmax(one_hot)
#print(one_hot_to_digit(y_train[i]))
yp=[]
#print(y_train[0])
#print(one_hot_to_digit(y_train[0]))
for i in range(60000):
    d=one_hot_to_digit(y_train[i])
    yp.append(d)
    if i ==200:
        print(i)
    

200


In [72]:
np.random.randn()

1.8752320475322997

In [83]:
def ini_params():
    w1=np.random.randn(10,784)*0.01#randn keeps the valoe in+-0.5 dont know how taht helps this is wrong
    
    b1=np.random.randn(10,1)*0.01#np.zeros((10,1))
    w2=np.random.randn(10,10)*0.01
    b2=np.random.randn(10,1)*0.01#np.zeros((10,1))#np.random.randn(10,1)*0.01
    return w1,b1,w2,b2
def one_hot_to_digit(one_hot):
    return np.argmax(one_hot)
def relu(x):
    return np.maximum(0,x)
def softmax(x):
    x = x - np.max(x, axis=0, keepdims=True)  
    return np.exp(x) / np.sum(np.exp(x), axis=0, keepdims=True)
#def softmax(x):
#    return np.exp(x)/np.sum(np.exp(x))
def frwd_props(w1,b1,w2,b2,x):
    z1=np.dot(w1,x.T)+b1#z1 is 10*784
    a1=relu(z1)
    z2=np.dot(w2,a1)+b2#10*784
    a2=softmax(z2)#softmax gives the probs of the number, we will pridict no. with higest probs
    return z1,z2,a1,a2
def derv_relu(x):
    return x>0#true or dalse so 1 0r 0
def back_props(a2,a1,z1,w2,y,x):
    dz2=a2-y.T#we use hot coded y itself
    dw2=np.dot(dz2,a1.T)/784
    db2=np.sum(dz2)/784
    dz1=np.dot(w2.T,dz2)*derv_relu(z1)
    dw1=np.dot(dz1,x)/784
    db1=np.sum(dz1)/784
    return dw1, db1,dw2,db2
def update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha):
    w2=w2-alpha*dw2
    b2=b2-alpha*db2
    w1=w1-alpha*dw1
    b1=b1-alpha*db1
    return w2,b2,w1,b1
def gradient_descent(x,y,itterations,alpha):
    w1,b1,w2,b2=ini_params()
    for i in range(itterations):
        z1,z2,a1,a2=frwd_props(w1,b1,w2,b2,x)
        dw1,db1,dw2,db2=back_props(a2,a1,z1,w2,y,x)
        w2,b2,w1,b1=update_params(w1,b1,w2,b2,dw1,db1,dw2,db2,alpha)
        if i%100==0:
            acc=accuracy(predictions(a2),yp)
            print("itteration : ",i)
            print("accuracy : ",acc*100)
            
    return w1,b1,w2,b2,a2
def predictions(x):
    return np.argmax(x,0)
def accuracy(preds,y):
    return np.sum(preds==y)/60000 


In [84]:
w1,b1,w2,b2,a2=gradient_descent(X_train,y_train,1000,0.01)
acc=accuracy(predictions(a2),yp)

#f1 score

print("accuracy : ",acc*100,"%")

itteration :  0
accuracy :  10.216666666666667
itteration :  100
accuracy :  79.28833333333334
itteration :  200
accuracy :  89.69500000000001
itteration :  300
accuracy :  90.69666666666667
itteration :  400
accuracy :  91.51333333333334
itteration :  500
accuracy :  91.79
itteration :  600
accuracy :  92.16166666666666
itteration :  700
accuracy :  92.47666666666666
itteration :  800
accuracy :  92.70166666666667
itteration :  900
accuracy :  90.7
accuracy :  92.04833333333333 %
